In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from classes import MixedNaiveBayes

from classes import CustomLogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import (
    auc,
    accuracy_score,
    roc_curve,
    confusion_matrix,
    classification_report
)
from sklearn.utils.class_weight import compute_class_weight

from helpers import cv_evaluate_model

import time

seed = 777

In [13]:
import pickle

(train_X_folds, train_y_folds, test_X_folds, test_y_folds, feature_names_folds, reverse_map_folds) = pickle.load(open('data/prepared_data.pkl', 'rb'))

### Baseline

In [14]:
print("=== Baseline ===")

alpha = 5
l1_ratio = 0.75

# Train Logistic Regression with L1 & L2 regularization
print("Training Logistic Regression with L1 & L2 regularization...")

logistic_regression_results = cv_evaluate_model(
    lambda: CustomLogisticRegression(
        alpha=alpha,
        l1_ratio=l1_ratio,
        max_iter=200,
        tol=1e-6,
        class_weight="balanced",
        random_state=seed
    ),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.predict_proba(X)[:, 1]
)

=== Baseline ===
Training Logistic Regression with L1 & L2 regularization...


In [ ]:
print("Baseline Model Performance Analysis")

print(
    f"\nAverage training time: {logistic_regression_results['training_time'][0]:.2f}s")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {logistic_regression_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {logistic_regression_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {logistic_regression_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {logistic_regression_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {logistic_regression_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {logistic_regression_results['test_pr_auc'][0]:.4f}")

cm = logistic_regression_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")

feature_importance = pd.Series(logistic_regression_results["results"][0]["model"].coef_, index=feature_names_folds[0])
feature_importance = feature_importance[feature_importance.abs().sort_values(ascending=False).index]

print(f"\nMost Important Features:")
print(feature_importance.head(20))

Baseline Model Performance Analysis

Average training time: 7.72

Metrics
Avg Accuracy:   0.6582
Avg Precision:  0.1800
Avg Recall:     0.5801
Avg F1:         0.2747
Avg ROC AUC:    0.6745
Avg PR AUC:     0.2111

Confusion Matrix
                Predicted       No Readmit      Readmit        
Actual          No Readmit      12089           6020           
                Readmit         994             1331           

Most Important Features:
discharge_disposition_id_11                  -4.215332
discharge_disposition_id_28                   1.689171
discharge_disposition_id_15                   1.364386
discharge_disposition_id_22                   1.351553
discharge_disposition_id_5                    1.030752
discharge_disposition_id_13                  -0.809176
medical_specialty_Otolaryngology             -0.775223
medical_specialty_Pediatrics-Endocrinology   -0.770679
medical_specialty_Hematology                  0.749855
discharge_disposition_id_9                    0.670702
di

### Naive Bayes

In [16]:
print("=== Naive Bayes ===")

# Train MixedNaiveBayes with balanced classes using cv_evaluate_model format
print("Training MixedNaiveBayes with balanced classes...")

naive_bayes_results = cv_evaluate_model(
    lambda: MixedNaiveBayes(class_weight='balanced'),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.predict_proba(X)[:, 1]
)

=== Naive Bayes ===
Training MixedNaiveBayes with balanced classes...


In [ ]:
print("Naive Bayes Model Performance Analysis")

print(
    f"\nAverage training time: {naive_bayes_results['training_time'][0]:.2f}s")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {naive_bayes_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {naive_bayes_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {naive_bayes_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {naive_bayes_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {naive_bayes_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {naive_bayes_results['test_pr_auc'][0]:.4f}")

cm = naive_bayes_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")

# Feature importance analysis for Naive Bayes using log likelihood
print(f"\nMost Important Features:")
model = naive_bayes_results["results"][0]["model"]
train_X_first_fold = train_X_folds[0]
feature_names_first_fold = feature_names_folds[0]

# Get log likelihood for each feature
log_likelihood = model.get_log_likelihood(train_X_first_fold)

# Calculate feature importance as the difference in log likelihood between classes
# Positive values indicate features that favor class 1 (readmission)
feature_importance_values = log_likelihood[:, 1] - log_likelihood[:, 0]

# Create pandas Series for consistent format
feature_importance = pd.Series(feature_importance_values, index=feature_names_first_fold)
feature_importance = feature_importance[feature_importance.abs().sort_values(ascending=False).index]

print(feature_importance.head(20))

Naive Bayes Model Performance Analysis

Average training time: 1.18

Metrics
Avg Accuracy:   0.6199
Avg Precision:  0.1446
Avg Recall:     0.4888
Avg F1:         0.2231
Avg ROC AUC:    0.5878
Avg PR AUC:     0.1544

Confusion Matrix
                Predicted       No Readmit      Readmit        
Actual          No Readmit      11473           6636           
                Readmit         1154            1171           

Most Important Features:
discharge_disposition_id_11                 -0.065009
insulin_yes                                  0.026994
admission_source_id_7                        0.017168
discharge_disposition_id_22                 -0.010460
metformin_yes                               -0.008313
age_[50-60)                                 -0.005458
medical_specialty_Cardiology                -0.003493
age_[70-80)                                  0.003474
admission_type_id_3                         -0.003086
discharge_disposition_id_5                  -0.002910
age_[80-9

### SVM

In [18]:
print("=== Linear SVM ===")

# Tunable parameters
max_iter = 1000  # Will be optimized in finetuning
learning_rate = 'optimal'  # Will be optimized in finetuning

# Train SGDClassifier with hinge loss (equivalent to SVM) using cv_evaluate_model format
print("Training SGDClassifier with hinge loss (Linear SVM)...")

linear_svm_results = cv_evaluate_model(
    lambda: SGDClassifier(
        loss='hinge',           # Hinge loss = SVM
        max_iter=max_iter,
        random_state=seed,
        class_weight='balanced',
        learning_rate=learning_rate
    ),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.decision_function(X)
)

=== Linear SVM ===
Training SGDClassifier with hinge loss (Linear SVM)...


In [19]:
print("Linear SVM Model Performance Analysis")

print(
    f"\nAverage training time: {linear_svm_results['training_time'][0]:.2f}s")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {linear_svm_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {linear_svm_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {linear_svm_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {linear_svm_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {linear_svm_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {linear_svm_results['test_pr_auc'][0]:.4f}")

cm = linear_svm_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")

# Feature importance analysis for Linear SVM (adjusted by standard deviation)
print(f"\nMost Important Features (Std-Adjusted):")

# Get the first fold data for standard deviation calculation
train_X_first_fold = train_X_folds[0]
feature_names_first_fold = feature_names_folds[0]

# Calculate standard deviation for each feature
feature_std = np.std(train_X_first_fold, axis=0)

# Get raw coefficients
raw_coef = linear_svm_results["results"][0]["model"].coef_[0]

# Calculate adjusted feature importance: coefficient * std_dev (preserve direction)
feature_importance_with_direction = raw_coef * feature_std

# Create pandas Series for consistent format with baseline
feature_importance = pd.Series(feature_importance_with_direction, index=feature_names_first_fold)
feature_importance = feature_importance[feature_importance.abs().sort_values(ascending=False).index]

print(f"\nMost Important Features:")
print(feature_importance.head(20))

Linear SVM Model Performance Analysis

Average training time: 5.32s

Metrics
Avg Accuracy:   0.6536
Avg Precision:  0.1778
Avg Recall:     0.5800
Avg F1:         0.2720
Avg ROC AUC:    0.6686
Avg PR AUC:     0.2069

Confusion Matrix
                Predicted       No Readmit      Readmit        
Actual          No Readmit      12496           5613           
                Readmit         1056            1269           

Most Important Features (Std-Adjusted):

Most Important Features:
number_inpatient               0.494645
discharge_disposition_id_11   -0.303617
discharge_disposition_id_22    0.233930
discharge_disposition_id_3     0.227592
discharge_disposition_id_5     0.158642
diag_PC7                       0.128290
diag_PC1                       0.123597
discharge_disposition_id_2     0.119308
diag_PC19                      0.112456
age_[80-90)                    0.107065
discharge_disposition_id_6     0.099763
admission_source_id_17        -0.098744
gender_Male x age_[60-70)   

### Kernel SVM

In [ ]:
print("=== RBF Kernel SVM ===")

# Tunable parameters
C = 1.0  # Will be optimized in finetuning
gamma = 'scale'  # Will be optimized in finetuning

# Train RBF SVM with balanced classes using cv_evaluate_model format
print("Training RBF SVM with balanced classes...")

rbf_svm_results = cv_evaluate_model(
    lambda: SVC(
        kernel='rbf',
        C=C,
        gamma=gamma,
        class_weight='balanced',
        random_state=seed
    ),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.decision_function(X)
)


In [ ]:
print("RBF SVM Model Performance Analysis")

print(
    f"\nAverage training time: {rbf_svm_results['training_time'][0]:.2f}s")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {rbf_svm_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {rbf_svm_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {rbf_svm_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {rbf_svm_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {rbf_svm_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {rbf_svm_results['test_pr_auc'][0]:.4f}")

cm = rbf_svm_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")


### Decision Tree

In [20]:
print("=== Decision Tree ===")

# Tunable parameters
max_depth = 10  # Will be optimized in finetuning
min_samples_split = 20  # Will be optimized in finetuning
min_samples_leaf = 10  # Will be optimized in finetuning
max_features = 'sqrt'  # Will be optimized in finetuning

# Train Decision Tree with balanced classes using cv_evaluate_model format
print("Training Decision Tree with balanced classes...")

decision_tree_results = cv_evaluate_model(
    lambda: DecisionTreeClassifier(
        random_state=seed,
        class_weight='balanced',
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features
    ),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.predict_proba(X)[:, 1]
)


=== Decision Tree ===
Training Decision Tree with balanced classes...


In [21]:
print("Decision Tree Model Performance Analysis")

print(
    f"\nAverage training time: {decision_tree_results['training_time'][0]:.2f}s")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {decision_tree_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {decision_tree_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {decision_tree_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {decision_tree_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {decision_tree_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {decision_tree_results['test_pr_auc'][0]:.4f}")

cm = decision_tree_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")

# Feature importance analysis
print(f"\nMost Important Features:")
feature_importance = pd.Series(decision_tree_results["results"][0]["model"].feature_importances_, index=feature_names_folds[0])
feature_importance = feature_importance[feature_importance.abs().sort_values(ascending=False).index]
print(feature_importance.head(20))


Decision Tree Model Performance Analysis

Average training time: 0.78s

Metrics
Avg Accuracy:   0.5978
Avg Precision:  0.1451
Avg Recall:     0.5261
Avg F1:         0.2262
Avg ROC AUC:    0.5859
Avg PR AUC:     0.1530

Confusion Matrix
                Predicted       No Readmit      Readmit        
Actual          No Readmit      9003            9106           
                Readmit         891             1434           

Most Important Features:
number_inpatient               0.144029
discharge_disposition_id_11    0.039930
diag_PC5                       0.029617
number_diagnoses               0.025843
diag_PC143                     0.020421
diag_PC88                      0.015053
diag_PC12                      0.012598
diag_PC54                      0.012260
diag_PC74                      0.011164
diag_PC26                      0.010982
diag_PC155                     0.010584
diag_PC66                      0.010102
diag_PC142                     0.009905
diag_PC105                

### Random Forest

In [22]:
print("=== Random Forest ===")

# Tunable parameters
n_estimators = 100  # Will be optimized in finetuning
max_depth = 10  # Will be optimized in finetuning
min_samples_split = 20  # Will be optimized in finetuning
min_samples_leaf = 10  # Will be optimized in finetuning
max_features = 'sqrt'  # Will be optimized in finetuning

# Train Random Forest with balanced classes using cv_evaluate_model format
print("Training Random Forest with balanced classes...")

random_forest_results = cv_evaluate_model(
    lambda: RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight='balanced',
        random_state=seed,
        n_jobs=-1
    ),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.predict_proba(X)[:, 1]
)


=== Random Forest ===
Training Random Forest with balanced classes...


In [23]:
print("Random Forest Model Performance Analysis")

print(
    f"\nAverage training time: {random_forest_results['training_time'][0]:.2f}")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {random_forest_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {random_forest_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {random_forest_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {random_forest_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {random_forest_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {random_forest_results['test_pr_auc'][0]:.4f}")

cm = random_forest_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")

# Feature importance analysis
print(f"\nMost Important Features:")
feature_importance = pd.Series(random_forest_results["results"][0]["model"].feature_importances_, index=feature_names_folds[0])
feature_importance = feature_importance[feature_importance.abs().sort_values(ascending=False).index]
print(feature_importance.head(20))

Random Forest Model Performance Analysis

Average training time: 9.51

Metrics
Avg Accuracy:   0.7468
Avg Precision:  0.1870
Avg Recall:     0.3790
Avg F1:         0.2504
Avg ROC AUC:    0.6408
Avg PR AUC:     0.1833

Confusion Matrix
                Predicted       No Readmit      Readmit        
Actual          No Readmit      14135           3974           
                Readmit         1412            913            

Most Important Features:
number_inpatient               0.096771
discharge_disposition_id_11    0.022389
discharge_disposition_id_22    0.020103
diag_PC1                       0.017991
number_emergency               0.016362
number_diagnoses               0.011200
num_medications                0.011043
diag_PC5                       0.010119
diag_PC0                       0.010033
time_in_hospital               0.008833
diag_PC19                      0.008094
discharge_disposition_id_3     0.007670
diag_PC68                      0.007475
diag_PC7                   

### Optimized Random Forest

In [24]:
print("=== XGBoost ===")

# Tunable parameters
n_estimators = 100  # Will be optimized in finetuning
max_depth = 6  # Will be optimized in finetuning
learning_rate = 0.1  # Will be optimized in finetuning
subsample = 0.8  # Will be optimized in finetuning
colsample_bytree = 0.8  # Will be optimized in finetuning

# Train XGBoost with balanced classes using cv_evaluate_model format
print("Training XGBoost with balanced classes...")

xgb_results = cv_evaluate_model(
    lambda: xgb.XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        scale_pos_weight=len(train_y_folds[0][train_y_folds[0]==0])/len(train_y_folds[0][train_y_folds[0]==1]),  # Handle class imbalance
        random_state=seed,
        n_jobs=-1,
        eval_metric='logloss'
    ),
    train_X_folds,
    train_y_folds,
    test_X_folds,
    test_y_folds,
    get_decision_score=lambda model, X: model.predict_proba(X)[:, 1]
)


=== XGBoost ===
Training XGBoost with balanced classes...


In [25]:
print("XGBoost Model Performance Analysis")

print(
    f"\nAverage training time: {xgb_results['training_time'][0]:.2f}")

print("\nMetrics")
print(
    f"{'Avg Accuracy:':<15} {xgb_results['test_accuracy'][0]:.4f}")
print(
    f"{'Avg Precision:':<15} {xgb_results['test_precision_score'][0]:.4f}")
print(
    f"{'Avg Recall:':<15} {xgb_results['test_recall_score'][0]:.4f}")
print(f"{'Avg F1:':<15} {xgb_results['test_f1'][0]:.4f}")
print(
    f"{'Avg ROC AUC:':<15} {xgb_results['test_roc_auc'][0]:.4f}")
print(
    f"{'Avg PR AUC:':<15} {xgb_results['test_pr_auc'][0]:.4f}")

cm = xgb_results["results"][0]["test_cm"]

print("\nConfusion Matrix")
print(f"{'':<15} {'Predicted':<15} {'No Readmit':<15} {'Readmit':<15}")
print(f"{'Actual':<15} {'No Readmit':<15} {cm[0,0]:<15} {cm[0,1]:<15}")
print(f"{'':<15} {'Readmit':<15} {cm[1,0]:<15} {cm[1,1]:<15}")

# Feature importance analysis
print(f"\nMost Important Features:")
feature_importance = pd.Series(xgb_results["results"][0]["model"].feature_importances_, index=feature_names_folds[0])
feature_importance = feature_importance[feature_importance.abs().sort_values(ascending=False).index]
print(feature_importance.head(20))


XGBoost Model Performance Analysis

Average training time: 2.70

Metrics
Avg Accuracy:   0.6989
Avg Precision:  0.1860
Avg Recall:     0.5023
Avg F1:         0.2714
Avg ROC AUC:    0.6642
Avg PR AUC:     0.2125

Confusion Matrix
                Predicted       No Readmit      Readmit        
Actual          No Readmit      13061           5048           
                Readmit         1141            1184           

Most Important Features:
number_inpatient                            0.024820
discharge_disposition_id_22                 0.018417
discharge_disposition_id_11                 0.013670
discharge_disposition_id_3                  0.011549
diag_PC1                                    0.009219
discharge_disposition_id_5                  0.008564
number_emergency                            0.008168
discharge_disposition_id_2                  0.005635
diag_PC140                                  0.005380
discharge_disposition_id_28                 0.005274
number_diagnoses       

### Comparison

In [26]:
# Create comparison table with all model results
print("=== Model Comparison Table ===")

# Collect all results
models_data = {
    'Logistic Regression': logistic_regression_results,
    'Naive Bayes': naive_bayes_results,
    'Linear SVM': linear_svm_results,
    # 'RBF SVM': rbf_svm_results,  # Commented out - takes too long to run
    'Decision Tree': decision_tree_results,
    'Random Forest': random_forest_results,
    'XGBoost': xgb_results
}

# Create comparison DataFrame
comparison_data = []
for model_name, results in models_data.items():
    comparison_data.append({
        'Model': model_name,
        'Training Time (s)': f"{results['training_time'][0]:.2f}",
        'Accuracy': f"{results['test_accuracy'][0]:.4f}",
        'Precision': f"{results['test_precision_score'][0]:.4f}",
        'Recall': f"{results['test_recall_score'][0]:.4f}",
        'F1 Score': f"{results['test_f1'][0]:.4f}",
        'ROC AUC': f"{results['test_roc_auc'][0]:.4f}",
        'PR AUC': f"{results['test_pr_auc'][0]:.4f}"
    })

# Add commented out RBF SVM row
comparison_data.append({
    'Model': '# RBF SVM (commented out - too slow)',
    'Training Time (s)': 'N/A',
    'Accuracy': 'N/A',
    'Precision': 'N/A',
    'Recall': 'N/A',
    'F1 Score': 'N/A',
    'ROC AUC': 'N/A',
    'PR AUC': 'N/A'
})

comparison_df = pd.DataFrame(comparison_data)

# Display the comparison table
print("\nModel Performance Comparison:")
print("=" * 100)
print(comparison_df.to_string(index=False))

# Find best performing models for each metric
print("\n" + "=" * 50)
print("BEST PERFORMING MODELS:")
print("=" * 50)

# Convert numeric columns for comparison (excluding the commented row)
numeric_data = comparison_df[comparison_df['Model'] != '# RBF SVM (commented out - too slow)'].copy()
for col in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC', 'PR AUC']:
    numeric_data[col] = numeric_data[col].astype(float)

for metric in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC', 'PR AUC']:
    best_idx = numeric_data[metric].idxmax()
    best_model = numeric_data.loc[best_idx, 'Model']
    best_score = numeric_data.loc[best_idx, metric]
    print(f"{metric:<12}: {best_model:<20} ({best_score:.4f})")

print("\n" + "=" * 50)
print("TRAINING TIME ANALYSIS:")
print("=" * 50)
fastest_idx = numeric_data['Training Time (s)'].astype(float).idxmin()
fastest_model = numeric_data.loc[fastest_idx, 'Model']
fastest_time = numeric_data.loc[fastest_idx, 'Training Time (s)']
print(f"Fastest Training: {fastest_model:<20} ({fastest_time}s)")

slowest_idx = numeric_data['Training Time (s)'].astype(float).idxmax()
slowest_model = numeric_data.loc[slowest_idx, 'Model']
slowest_time = numeric_data.loc[slowest_idx, 'Training Time (s)']
print(f"Slowest Training: {slowest_model:<20} ({slowest_time}s)")


=== Model Comparison Table ===

Model Performance Comparison:
                               Model Training Time (s) Accuracy Precision Recall F1 Score ROC AUC PR AUC
                 Logistic Regression              7.72   0.6582    0.1800 0.5801   0.2747  0.6745 0.2111
                         Naive Bayes              1.18   0.6199    0.1446 0.4888   0.2231  0.5878 0.1544
                          Linear SVM              5.32   0.6536    0.1778 0.5800   0.2720  0.6686 0.2069
                       Decision Tree              0.78   0.5978    0.1451 0.5261   0.2262  0.5859 0.1530
                       Random Forest              9.51   0.7468    0.1870 0.3790   0.2504  0.6408 0.1833
                             XGBoost              2.70   0.6989    0.1860 0.5023   0.2714  0.6642 0.2125
# RBF SVM (commented out - too slow)               N/A      N/A       N/A    N/A      N/A     N/A    N/A

BEST PERFORMING MODELS:
Accuracy    : Random Forest        (0.7468)
Precision   : Random Forest  